## Problem 1

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm
import inspect

In [3]:
# Given information
S0 = 151.03
X = 165
r = 0.0425
coupon = 0.0053
b = r - coupon
ivol = 0.2
current_date = datetime(2022,3,13)
exp_date = datetime(2022,4,15)

In [10]:
# Function to calculate time to maturity using calendar days
def calculate_ttm(curr_date, expire_date):
    days_to_expiration = (expire_date - curr_date).days
    return days_to_expiration / 365

# Function to calculate d1 and d2
def calculate_d1_d2(S0, X, b, T, ivol):
    d1 = (np.log(S0 / X) + (b + ivol**2 / 2) * T) / (ivol * np.sqrt(T))
    d2 = d1 - ivol * np.sqrt(T)
    return d1, d2

T = calculate_ttm(current_date, exp_date)

# Closed form greeks functions for GBSM
def delta_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        delta = np.exp((b-r)*T) * norm.cdf(d1)
    else:
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
    return delta

def gamma_gbsm(S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    gamma = (norm.pdf(d1)*np.exp((b-r)*T)) / (S0 * ivol *np.sqrt(T))
    return gamma

def vega_gbsm(S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    vega = S0 * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    return vega

def theta_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        theta = -(S0*np.exp((b-r)*T)*norm.pdf(d1)*ivol)/(2*np.sqrt(T))\
                -(b-r)*S0*np.exp((b-r)*T)*norm.cdf(d1)\
                -r*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        theta = -(S0*np.exp((b-r)*T)*norm.pdf(d1)*ivol)/(2*np.sqrt(T))\
                +(b-r)*S0*np.exp((b-r)*T)*norm.cdf(-d1)\
                +r*X*np.exp(-r*T)*norm.cdf(-d2)
    return theta

def rho_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        rho = T*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        rho = -T*X*np.exp(-r*T)*norm.cdf(-d2)
    return rho

def carry_rho_gbsm(option_type, S0, X, T, ivol, b, r):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        carry_rho = T*S0*np.exp((b-r)*T)*norm.cdf(d1)
    else:
        carry_rho = -T*S0*np.exp((b-r)*T)*norm.cdf(-d1)
    return carry_rho

# Results
print("Closed form greeks for GBSM:")
print("Call's Delta is ", delta_gbsm('Call', S0, X, T, ivol, b, r))
print("Put's Delta is ", delta_gbsm('Put', S0, X, T, ivol, b, r))

print("Call's Gamma is ", gamma_gbsm(S0, X, T, ivol, b, r))
print("Put's Gamma is ", gamma_gbsm(S0, X, T, ivol, b, r))

print("Call's Vega is ", vega_gbsm(S0, X, T, ivol, b, r))
print("Put's Vega is ", vega_gbsm(S0, X, T, ivol, b, r))

print("Call's Theta is ", theta_gbsm("Call", S0, X, T, ivol, b, r))
print("Put's Theta is ", theta_gbsm("Put", S0, X, T, ivol, b, r))       

print("Call's Rho is ", rho_gbsm("Call", S0, X, T, ivol, b, r))
print("Put's Rho is ", rho_gbsm("Put", S0, X, T, ivol, b, r)) 

print("Call's Carry Rho is ", carry_rho_gbsm( "Call", S0, X, T, ivol, b, r))
print("Put's Carry Rho is ", carry_rho_gbsm( "Put", S0, X, T, ivol, b, r)) 

Closed form greeks for GBSM:
Call's Delta is  0.08297130333914773
Put's Delta is  -0.9165496333661425
Call's Gamma is  0.016822916101852648
Put's Gamma is  0.016822916101852648
Call's Vega is  6.938710929513443
Put's Vega is  6.938710929513443
Call's Theta is  -8.126522359668838
Put's Theta is  -1.9409914783019557
Call's Rho is  1.102593915636819
Put's Rho is  -13.75800312273579
Call's Carry Rho is  1.132953825011723
Put's Carry Rho is  -12.515271800549371


In [11]:
def gbsm(option_type, S0, X, T, ivol, r, b):
    d1, d2 = calculate_d1_d2(S0, X, b, T, ivol)
    if option_type == "Call":
        call_value = S0 * np.exp((b-r)*T)*norm.cdf(d1) - X * np.exp(-r*T)*norm.cdf(d2)
        return call_value
    else:
        put_value = X * np.exp(-r*T)*norm.cdf(-d2) - S0 * np.exp((b-r)*T)*norm.cdf(-d1)
        return put_value

# Implement a finite diference derivative calculation
def first_order_derivative(function, x, delta):
    result = (function(x+delta) - function(x-delta)) / (2*delta)
    return result

def second_order_devirvative(function, x, delta):
    result = (function(x+delta) + function(x-delta) - 2*function(x)) / (delta**2)
    return result

def cal_derivative_wrt_one(function, order, object_arg, delta = 1e-3):
    all_args = list(inspect.signature(function).parameters.keys())
    orders_dic = {1:first_order_derivative, 2:second_order_devirvative}

    def cal_derivative(*args, **kwargs):
        args_dic = dict(list(zip(all_args, args)) + list(kwargs.items()))
        value_arg = args_dic.pop(object_arg)

        def trans_into_one_arg(x):
            all_args = {object_arg:x, **args_dic}
            return function(**all_args)
        return orders_dic[order](trans_into_one_arg, value_arg, delta)
    return cal_derivative

# Results
print("Finite difference greeks:")
gbsm_delta = cal_derivative_wrt_one(gbsm, 1, 'S0')
print("Call's Delta is ", gbsm_delta("Call", S0, X, T, ivol, r, b))
print("Put's Delta is ", gbsm_delta("Put", S0, X, T, ivol, r, b)) 
gbsm_gamma = cal_derivative_wrt_one(gbsm, 2 ,'S0')
print("Call's Gamma is ", gbsm_gamma("Call", S0, X, T, ivol, r, b))
print("Put's Gamma is ", gbsm_gamma("Put", S0, X, T, ivol, r, b)) 
gbsm_vega = cal_derivative_wrt_one(gbsm, 1 ,'ivol')
print("Call's Vega is ", gbsm_vega("Call", S0, X, T, ivol, r, b))
print("Put's Vega is ", gbsm_vega("Put", S0, X, T, ivol, r, b)) 
gbsm_theta = cal_derivative_wrt_one(gbsm, 1 ,'T')
print("Call's Theta is ", -gbsm_theta("Call", S0, X, T, ivol, r, b))
print("Put's Theta is ", -gbsm_theta("Put", S0, X, T, ivol, r, b))
gbsm_rho = cal_derivative_wrt_one(gbsm, 1 ,'r')
print("Call's Rho is ", gbsm_rho("Call", S0, X, T, ivol, r, b))
print("Put's Rho is ", gbsm_rho("Put", S0, X, T, ivol, r, b))
gbsm_carry_rho = cal_derivative_wrt_one(gbsm, 1 ,'b')
print("Call's Carry Rho is ", gbsm_carry_rho( "Call", S0, X, T, ivol, r, b))
print("Put's Carry Rho is ", gbsm_carry_rho( "Put", S0, X, T, ivol, r, b))

Finite difference greeks:
Call's Delta is  0.08297130374668171
Put's Delta is  -0.9165496329472944
Call's Gamma is  0.016822911064195978
Put's Gamma is  0.016822951920403284
Call's Vega is  6.938653056250743
Put's Vega is  6.93865305626673
Call's Theta is  -8.126308803761084
Put's Theta is  -1.9407779203106656
Call's Rho is  -0.030359909417576603
Put's Rho is  -1.2427313238703164
Call's Carry Rho is  1.1329550097096686
Put's Carry Rho is  -12.515270634423814
